In [15]:
from skillcorner.client import SkillcornerClient
from dotenv import load_dotenv
import requests
import pandas as pd
import os
import json

In [3]:
load_dotenv()
username = os.getenv('U')
password = os.getenv('P')

In [4]:
client = SkillcornerClient(username=username, password=password)

In [16]:
seasons = client.get_seasons()
seasons

[{'id': 95, 'start_year': 2024, 'end_year': 2025, 'name': '2024/2025'},
 {'id': 29, 'start_year': 2024, 'end_year': 2024, 'name': '2024'},
 {'id': 28, 'start_year': 2023, 'end_year': 2024, 'name': '2023/2024'},
 {'id': 22, 'start_year': 2023, 'end_year': 2023, 'name': '2023'},
 {'id': 21, 'start_year': 2022, 'end_year': 2023, 'name': '2022/2023'},
 {'id': 20, 'start_year': 2022, 'end_year': 2022, 'name': '2022'},
 {'id': 8, 'start_year': 2021, 'end_year': 2022, 'name': '2021/2022'},
 {'id': 17, 'start_year': 2021, 'end_year': 2021, 'name': '2021'},
 {'id': 7, 'start_year': 2020, 'end_year': 2021, 'name': '2020/2021'},
 {'id': 16, 'start_year': 2020, 'end_year': 2020, 'name': '2020'},
 {'id': 6, 'start_year': 2019, 'end_year': 2020, 'name': '2019/2020'},
 {'id': 15, 'start_year': 2019, 'end_year': 2019, 'name': '2019'},
 {'id': 5, 'start_year': 2018, 'end_year': 2019, 'name': '2018/2019'},
 {'id': 14, 'start_year': 2018, 'end_year': 2018, 'name': '2018'},
 {'id': 4, 'start_year': 2017, 

In [18]:
comp_editions=pd.DataFrame(client.get_competition_editions(params={'user':'true'}))
comp_editions=comp_editions[['id','name']]
comp_editions

,id,name
0,574,ENG - FA Women`s Super League - 2023/2024
1,800,USA - NWSL - 2024


In [5]:
FA_matches = client.get_matches(params={'competition_edition': 574})

In [18]:
len(FA_matches)

132

In [19]:
NWSL_matches = client.get_matches(params={'competition_edition': 800})

In [20]:
len(NWSL_matches)

189

In [6]:
match_id = FA_matches[1]['id']

In [32]:
match_data = client.get_match(match_id=match_id)
match_data

{'id': 1541631,
 'home_team_score': 5,
 'away_team_score': 0,
 'date_time': '2024-05-18T14:00:00Z',
 'stadium': {'id': 1826,
  'name': 'Meadow Park',
  'city': 'Borehamwood',
  'capacity': 4502},
 'home_team': {'id': 2026,
  'name': 'Arsenal Football Club, Women',
  'short_name': 'Arsenal',
  'acronym': 'ARS'},
 'home_team_kit': {'id': 5447,
  'team_id': 2026,
  'season': {'id': 6,
   'start_year': 2019,
   'end_year': 2020,
   'name': '2019/2020'},
  'name': 'Home',
  'jersey_color': '#ff0000',
  'number_color': '#ffffff'},
 'away_team': {'id': 2025,
  'name': 'Brighton and Hove Albion, Women',
  'short_name': 'Brighton',
  'acronym': 'BHA'},
 'away_team_kit': {'id': 10108,
  'team_id': 2025,
  'season': {'id': 29, 'start_year': 2024, 'end_year': 2024, 'name': '2024'},
  'name': 'away',
  'jersey_color': '#000000',
  'number_color': '#ffffff'},
 'home_team_coach': None,
 'away_team_coach': None,
 'competition_edition': {'id': 574,
  'competition': {'id': 127,
   'area': 'ENG',
   'nam

In [42]:
match_id

1541631

In [7]:
%%time
match_tracking_data = client.get_match_tracking_data(match_id=match_id, params={'data_version': 3})

CPU times: total: 2.25 s
Wall time: 9.31 s


In [9]:
with open("test.jsonl", "w") as jsonl_file:
    for obj in match_tracking_data:
        jsonl_file.write(json.dumps(obj) + "\n")

In [57]:
%%time
client.save_match_tracking_data(match_id=match_id, filepath=f'data/match_tracking_data_{match_id}.jsonl', params={'data_version': 3})

CPU times: total: 1.7 s
Wall time: 12.6 s


In [11]:
FA_match_ids = [m['id'] for m in FA_matches]

In [13]:
%%time
for match_id in FA_match_ids:
    file_path = f"data/FA/tracking/{match_id}.jsonl"
    if os.path.exists(file_path):
        continue

    print(f"Downloading {match_id}")
    match_tracking_data = client.get_match_tracking_data(match_id=match_id, params={'data_version': 3})
    
    # Write the data to a JSONL file
    with open(file_path, "w") as jsonl_file:
        for obj in match_tracking_data:
            jsonl_file.write(json.dumps(obj) + "\n")

CPU times: total: 12min 13s
Wall time: 26min 30s


In [21]:
for match_id in FA_match_ids:
    file_path = f"data/FA/match/{match_id}.json"
    if os.path.exists(file_path):
        continue

    print(f"Downloading {match_id}")
    match_data = match_data = client.get_match(match_id=match_id)
    
    with open(file_path, "w") as json_file:
        json.dump(match_data, json_file, indent=4) 

In [23]:
client.save_match(match_id=match_id, filepath="match_data.json", params={'matching':'wyscout'})

In [28]:
NWSL_match_ids = [m['id'] for m in NWSL_matches]

In [ ]:
for match_id in NWSL_match_ids:
    print(f'downloading {match_id}')
    client.save_match_tracking_data(match_id=match_id, filepath=f'data/NWSL/tracking/{match_id}.jsonl', params={'data_version': 3})

downloading 1875301
downloading 1864049
downloading 1862012
downloading 1850047
downloading 1850048
downloading 1848387
downloading 1846146
downloading 1837091
downloading 1836280
downloading 1833678
downloading 1833679
downloading 1835317
downloading 1831516
downloading 1832187
downloading 1804043
downloading 1804044
downloading 1804045
downloading 1802045
downloading 1801081
downloading 1801082
downloading 1804046
downloading 1788410
downloading 1787551
downloading 1790032
downloading 1785793
downloading 1785794
downloading 1784960
downloading 1783371
downloading 1775179
downloading 1767946
downloading 1767947
downloading 1769006
downloading 1769007
downloading 1765941
downloading 1765051
downloading 1752890
downloading 1752891
downloading 1749439
downloading 1774446
downloading 1749441
downloading 1748372
downloading 1747288
downloading 1737687
downloading 1737569
downloading 1731381
downloading 1731382
downloading 1731383
downloading 1728459
downloading 1728460
downloading 1721211


In [16]:
player_info_request=requests.get(f'https://skillcorner.com/api/match/{match_id}/?lang=en',
                                 auth=(username,password))

In [19]:
player_info_request.text.to_dict()

AttributeError: 'str' object has no attribute 'to_dict'